In [ ]:
# Give instructions for your agent
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write short, concise cold emails that are likely to get a response."

In [ ]:
# Import libraries to create your agents
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

In [ ]:
# Create your agents
sales_agent1 = Agent(
    name="Serious Sales Agent", 
    instructions=instructions1, 
    model='gpt-4o-mini'
    )

sales_agent2 = Agent(
    name="Witty Sales Agent", 
    instructions=instructions2, 
    model='gpt-4o-mini'
    )

sales_agent3 = Agent(
    name="Busy Sales Agent", 
    instructions=instructions3, 
    model='gpt-4o-mini'
    )
    

In [ ]:
# Test using a single sales agent 1
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)


In [11]:
import asyncio

In [ ]:
# You can launch multiple agents in parallel
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    for output in outputs:
        print(output + "\n\n")
        

https://platform.openai.com/traces

In [ ]:
# Use tools to enable some conditional logic

# First notice that you can include tool functions in the agent

sales_agent1

Agent(name='Serious Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [26]:
# To include tool in tools=[] of the agent
# Wrap your function with the decorator
# @function_tool

from sendgrid import from_email, sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import os

def send_email(body: str):
    """ Send out an email with the give body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = 'krisdas8@gmail.com'
    to_email='krisdas8@gmail.com'
    content = Content(mime_type='text/plain', content=body)
    mail = Mail(from_email=from_email, to_emails=to_email, subject="Sales email", plain_text_content="body").get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}


In [28]:
send_email(body="This is a test email")

UnauthorizedError: HTTP Error 401: Unauthorized